# Dependencies

In [4]:
import pandas as pd

# Functions

In [2]:
# function to determine drugs that appear in both the filtered DrugBank set and MedChemExpress screening library
# searching drug names because not all drugs in DrugBank have listed CAS number

def filt_MCE(db_set,MCE_set):
    
    print("Size of DrugBank set:",db_set.shape[0])
    print("Size of MCE set:",MCE_set.shape[0])
    
    count = 0
    
    db_set = db_set.reset_index()
    MCE_set = MCE_set.reset_index()
    
    db_filt = pd.DataFrame(columns = list(db_set.columns))
    MCE_filt = pd.DataFrame(columns = list(MCE_set.columns))

    # searching for drug names that overlap in DrugBank and MedChemExpress screening library
    for index, row in db_set.iterrows():
        print(index+1,'out of',db_set.shape[0])
        # some long chemical names resulted in an error during string comparison and were therefore excluded
        if len(row['name']) < 40:
            indices = [i for i,s in enumerate(MCE_set['Product Name']) if row['name'] in MCE_set.loc[i]['Product Name']]
            if len(indices) == 0:
                indices += [j for j,t in enumerate(MCE_set['Synonyms']) if row['name'] in MCE_set.loc[j]['Synonyms']]
            if len(indices) != 0:
                temp = MCE_set.loc[indices]
                MCE_filt = MCE_filt.append(MCE_set.loc[temp.index[temp['Product Name'] == min(temp["Product Name"],key=len)]])
                db_filt = db_filt.append(row)
                count += 1;
        else:
            print("error: ",row["name"])
    print(count)
    
    # returns DataFrames with columns from original DrugBank and MedChemExpress DataFrames with overlapping drugs
    return db_filt, MCE_filt

# Reading input files

In [5]:
# read dataframe with DrugBank data
drugbank_df = pd.read_pickle('Data_files/drugbank_df.pkl')

db_sm = drugbank_df[drugbank_df["type"]=="small molecule"]
# extract drugs with logP > 3
db_sm_L = db_sm[db_sm["logP"].astype(float)>3]
# extract drugs with pKa of strongest basic group greater than 4 and less than 11
db_sm_LP = db_sm_L[(db_sm_L["cb_pKa"].astype(float)>4) & (db_sm_L["cb_pKa"].astype(float)<11)]
# extract drugs with physiological charge of >= 0
db_sm_LPC = db_sm_LP[(db_sm_LP["Phys Chg"].astype(float)>=0)]
# print selected drugs
db_sm_LPC.shape

(1588, 13)

In [7]:
# Import drugs from MCE Bioactive Compound Plus library
MCE_plus = pd.read_csv('Data_files/MCE_screening_plus.csv',keep_default_na=False)
print(MCE_plus.shape)
MCE_plus_A = MCE_plus[MCE_plus["Part"]=="Part A"]
print(MCE_plus_A.shape)
MCE_plus_A_10 = MCE_plus_A[MCE_plus_A["Concentration"]=="10mM"]
print(MCE_plus_A_10.shape)

# Choosing drugs from Part A in 10 mM DMSO solutions to facilitate ease of screening
MCE_plus_A_10

(14614, 16)
(12935, 16)
(12639, 16)


,Part,Plate,Well,Catalog Number,Product Name,Synonyms,CAS Number,M. Wt,Target,Pathway,Biological Activity,Formula,Solubility,Concentration,Research Area,Clinical Information
0,Part A,HY-L001P-Part A-1,A02,HY-100001,SKF-96365 (hydrochloride),,130495-35-1,402.91,Apoptosis; Autophagy; CRAC Channel; Potassium ...,Apoptosis; Autophagy; Membrane Transporter/Ion...,SKF-96365 hydrochloride is a potent TRP channe...,C22H27ClN2O3,H2O : 100 mg/mL (248.19 mM; Need ultrasonic); ...,10mM,Cancer; Cardiovascular Disease,No Development Reported
1,Part A,HY-L001P-Part A-1,A03,HY-100002,ML162,,1035072-16-2,477.40,Ferroptosis; Glutathione Peroxidase,Apoptosis; Metabolic Enzyme/Protease,ML162 is a covalent glutathione peroxidase 4 (...,C23H22Cl2N2O3S,DMSO : 100 mg/mL (209.47 mM; Need ultrasonic),10mM,Cancer,No Development Reported
2,Part A,HY-L001P-Part A-1,A04,HY-100003,ML-210,,1360705-96-9,475.32,Ferroptosis; Glutathione Peroxidase,Apoptosis; Metabolic Enzyme/Protease,ML-210 is a selective and covalent glutathione...,C22H20Cl2N4O4,DMSO : 25 mg/mL (52.60 mM; Need ultrasonic),10mM,Cancer,No Development Reported
3,Part A,HY-L001P-Part A-1,A05,HY-100004,Fumarate hydratase-IN-1,,1644060-37-6,446.54,Mitochondrial Metabolism,Metabolic Enzyme/Protease,Fumarate hydratase-IN-1 (compound 2) is a cell...,C27H30N2O4,DMSO : 50 mg/mL (111.97 mM; Need ultrasonic); ...,10mM,Cancer,No Development Reported
4,Part A,HY-L001P-Part A-1,A06,HY-100006A,MRT68921 (dihydrochloride),,2080306-21-2,507.50,ULK,Autophagy,MRT68921 dihydrochloride is a potent inhibitor...,C25H36Cl2N6O,H2O : 31.25 mg/mL (61.58 mM; Need ultrasonic);...,10mM,Cancer,No Development Reported
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12930,Part A,HY-L001P-Part A-162,F02,HY-Z0031,"(R)-(-)-1,2-Propanediol",,4254-14-2,76.09,Endogenous Metabolite; Others,Metabolic Enzyme/Protease; Others,"(R)-(-)-1,2-Propanediol is a (R)-enantiomer of...",C3H8O2,DMSO : 100 mg/mL (1314.23 mM; Need ultrasonic),10mM,Metabolic Disease,No Development Reported
12931,Part A,HY-L001P-Part A-162,F03,HY-Z0283,Benzamide,NSC-3114; Benzenecarboxamide; Phenylamide,55-21-0,121.14,Endogenous Metabolite; PARP,Cell Cycle/DNA Damage; Epigenetics; Metabolic ...,Benzamide inhibits poly(ADP-ribose) polymerase...,C7H7NO,DMSO : 120 mg/mL (990.59 mM; Need ultrasonic),10mM,Others,No Development Reported
12932,Part A,HY-L001P-Part A-162,F04,HY-Z0478,(-)-Limonene,(S)-(-)-Limonene,5989-54-8,136.23,Others,Others,(-)-Limonene ((S)-(-)-Limonene) is a monoterpe...,C10H16,H2O : 50 mg/mL (367.03 mM; Need ultrasonic); D...,10mM,Cardiovascular Disease,Launched
12933,Part A,HY-L001P-Part A-162,F05,HY-Z0548,"Methyl 3,4-dihydroxybenzoate",Protocatechuic acid methyl ester; Methyl proto...,2150-43-8,168.15,Apoptosis; Keap1-Nrf2,Apoptosis; NF-κB,"Methyl 3,4-dihydroxybenzoate (Protocatechuic a...",C8H8O4,DMSO : ≥ 100 mg/mL (594.71 mM),10mM,Cancer; Inflammation/Immunology,No Development Reported


# Drug selection for purchase

In [12]:
#determining which small molecules from DrugBank are in the set of purchasable compounds from MedChemExpress
# from "Part A" of the screening library at 10 mM in DMSO
db_all_full_plus_A_10, MCE_all_full_plus_A_10 = filt_MCE(db_sm,MCE_plus_A_10)

Size of DrugBank set: 11834
Size of MCE set: 12639
1 out of 11834
2 out of 11834
3 out of 11834
4 out of 11834
5 out of 11834
6 out of 11834
7 out of 11834
8 out of 11834
9 out of 11834
10 out of 11834
11 out of 11834
12 out of 11834
13 out of 11834
14 out of 11834
15 out of 11834
16 out of 11834
17 out of 11834
18 out of 11834
19 out of 11834
20 out of 11834
21 out of 11834
22 out of 11834
23 out of 11834
24 out of 11834
25 out of 11834
26 out of 11834
27 out of 11834
28 out of 11834
29 out of 11834
30 out of 11834
31 out of 11834
32 out of 11834
33 out of 11834
34 out of 11834
35 out of 11834
36 out of 11834
37 out of 11834
38 out of 11834
39 out of 11834
40 out of 11834
41 out of 11834
42 out of 11834
43 out of 11834
44 out of 11834
45 out of 11834
46 out of 11834
47 out of 11834
48 out of 11834
49 out of 11834
50 out of 11834
51 out of 11834
52 out of 11834
53 out of 11834
54 out of 11834
55 out of 11834
56 out of 11834
57 out of 11834
58 out of 11834
59 out of 11834
60 out of 1183

487 out of 11834
488 out of 11834
489 out of 11834
490 out of 11834
491 out of 11834
492 out of 11834
493 out of 11834
494 out of 11834
495 out of 11834
496 out of 11834
497 out of 11834
498 out of 11834
499 out of 11834
500 out of 11834
501 out of 11834
502 out of 11834
503 out of 11834
504 out of 11834
505 out of 11834
506 out of 11834
507 out of 11834
508 out of 11834
509 out of 11834
510 out of 11834
511 out of 11834
512 out of 11834
513 out of 11834
514 out of 11834
515 out of 11834
516 out of 11834
517 out of 11834
518 out of 11834
519 out of 11834
520 out of 11834
521 out of 11834
522 out of 11834
523 out of 11834
524 out of 11834
525 out of 11834
526 out of 11834
527 out of 11834
528 out of 11834
529 out of 11834
530 out of 11834
531 out of 11834
532 out of 11834
533 out of 11834
534 out of 11834
535 out of 11834
536 out of 11834
537 out of 11834
538 out of 11834
539 out of 11834
540 out of 11834
541 out of 11834
542 out of 11834
543 out of 11834
544 out of 11834
545 out of 118

969 out of 11834
970 out of 11834
971 out of 11834
972 out of 11834
973 out of 11834
974 out of 11834
975 out of 11834
976 out of 11834
977 out of 11834
978 out of 11834
979 out of 11834
980 out of 11834
981 out of 11834
982 out of 11834
983 out of 11834
984 out of 11834
985 out of 11834
986 out of 11834
987 out of 11834
988 out of 11834
989 out of 11834
990 out of 11834
991 out of 11834
992 out of 11834
993 out of 11834
994 out of 11834
995 out of 11834
996 out of 11834
997 out of 11834
998 out of 11834
999 out of 11834
1000 out of 11834
1001 out of 11834
1002 out of 11834
1003 out of 11834
1004 out of 11834
1005 out of 11834
1006 out of 11834
1007 out of 11834
1008 out of 11834
1009 out of 11834
1010 out of 11834
1011 out of 11834
1012 out of 11834
1013 out of 11834
1014 out of 11834
1015 out of 11834
1016 out of 11834
1017 out of 11834
1018 out of 11834
1019 out of 11834
1020 out of 11834
1021 out of 11834
1022 out of 11834
1023 out of 11834
1024 out of 11834
1025 out of 11834
1026 

1408 out of 11834
1409 out of 11834
1410 out of 11834
1411 out of 11834
1412 out of 11834
1413 out of 11834
1414 out of 11834
1415 out of 11834
1416 out of 11834
1417 out of 11834
1418 out of 11834
1419 out of 11834
1420 out of 11834
1421 out of 11834
1422 out of 11834
1423 out of 11834
1424 out of 11834
1425 out of 11834
1426 out of 11834
1427 out of 11834
1428 out of 11834
1429 out of 11834
1430 out of 11834
1431 out of 11834
1432 out of 11834
1433 out of 11834
1434 out of 11834
1435 out of 11834
1436 out of 11834
1437 out of 11834
1438 out of 11834
1439 out of 11834
1440 out of 11834
1441 out of 11834
1442 out of 11834
1443 out of 11834
1444 out of 11834
1445 out of 11834
1446 out of 11834
1447 out of 11834
1448 out of 11834
1449 out of 11834
1450 out of 11834
error:  5-O-phosphono-alpha-D-ribofuranosyl diphosphate
1451 out of 11834
1452 out of 11834
error:  2-Oxy-4-Hydroxy-5-(2-Hydrazinopyridine)Phenylalanine
1453 out of 11834
error:  2,2-Dimethylthiazolidine-4-Carboxylic Acid;(Dmt

1647 out of 11834
1648 out of 11834
1649 out of 11834
1650 out of 11834
error:  (2S,3S,8S,9S)-3-amino-9-methoxy-2,6,8-trimethyl-10-phenyldeca-4,6-dienoic acid
1651 out of 11834
1652 out of 11834
1653 out of 11834
1654 out of 11834
1655 out of 11834
error:  5-(Hydroxy-Methyl-Amino)-3-Methyl-Pyrrolidine-2-Carboxylic Acid
1656 out of 11834
1657 out of 11834
1658 out of 11834
error:  [1-(4-Fluorobenzyl)Cyclobutyl]Methyl (1s)-1-[Oxo(1h-Pyrazol-5-Ylamino)Acetyl]Pentylcarbamate
1659 out of 11834
error:  4-CDP-2-C-methyl-D-erythritol 2-phosphate
1660 out of 11834
1661 out of 11834
1662 out of 11834
1663 out of 11834
1664 out of 11834
error:  5'-Guanosine-Diphosphate-Monothiophosphate
1665 out of 11834
error:  3-(6-Aminopyridin-3-Yl)-N-Methyl-N-[(1-Methyl-1h-Indol-2-Yl)Methyl]Acrylamide
1666 out of 11834
1667 out of 11834
error:  Glycyl-L-a-Aminopimelyl-E-(D-2-Aminoethyl)Phosphonate
1668 out of 11834
1669 out of 11834
error:  [1-(1-Benzyl-3-Hydroxy-2-Oxo-Propylcarbamoyl)-2-Phenyl-Ethyl]-Carbami

...

9919 out of 11834
9920 out of 11834
9921 out of 11834
9922 out of 11834
9923 out of 11834
9924 out of 11834
9925 out of 11834
9926 out of 11834
9927 out of 11834
9928 out of 11834
9929 out of 11834
9930 out of 11834
9931 out of 11834
9932 out of 11834
9933 out of 11834
9934 out of 11834
9935 out of 11834
9936 out of 11834
9937 out of 11834
9938 out of 11834
9939 out of 11834
9940 out of 11834
9941 out of 11834
9942 out of 11834
9943 out of 11834
9944 out of 11834
9945 out of 11834
9946 out of 11834
9947 out of 11834
9948 out of 11834
9949 out of 11834
9950 out of 11834
9951 out of 11834
9952 out of 11834
9953 out of 11834
9954 out of 11834
9955 out of 11834
9956 out of 11834
9957 out of 11834
9958 out of 11834
9959 out of 11834
9960 out of 11834
9961 out of 11834
9962 out of 11834
9963 out of 11834
9964 out of 11834
9965 out of 11834
9966 out of 11834
9967 out of 11834
9968 out of 11834
9969 out of 11834
9970 out of 11834
9971 out of 11834
9972 out of 11834
9973 out of 11834
9974 out o

10352 out of 11834
10353 out of 11834
10354 out of 11834
10355 out of 11834
10356 out of 11834
10357 out of 11834
10358 out of 11834
10359 out of 11834
10360 out of 11834
10361 out of 11834
10362 out of 11834
10363 out of 11834
10364 out of 11834
10365 out of 11834
10366 out of 11834
10367 out of 11834
10368 out of 11834
10369 out of 11834
10370 out of 11834
10371 out of 11834
10372 out of 11834
10373 out of 11834
10374 out of 11834
10375 out of 11834
10376 out of 11834
10377 out of 11834
10378 out of 11834
10379 out of 11834
10380 out of 11834
10381 out of 11834
10382 out of 11834
10383 out of 11834
10384 out of 11834
10385 out of 11834
error:  2-Methoxy-6-{(E)-[(4-methylphenyl)imino]methyl}phenol
10386 out of 11834
10387 out of 11834
10388 out of 11834
10389 out of 11834
10390 out of 11834
10391 out of 11834
10392 out of 11834
10393 out of 11834
10394 out of 11834
10395 out of 11834
10396 out of 11834
10397 out of 11834
10398 out of 11834
10399 out of 11834
10400 out of 11834
10401 o

10737 out of 11834
10738 out of 11834
10739 out of 11834
10740 out of 11834
10741 out of 11834
10742 out of 11834
10743 out of 11834
10744 out of 11834
10745 out of 11834
10746 out of 11834
10747 out of 11834
10748 out of 11834
10749 out of 11834
10750 out of 11834
10751 out of 11834
10752 out of 11834
10753 out of 11834
10754 out of 11834
10755 out of 11834
10756 out of 11834
10757 out of 11834
10758 out of 11834
10759 out of 11834
10760 out of 11834
10761 out of 11834
10762 out of 11834
10763 out of 11834
10764 out of 11834
10765 out of 11834
10766 out of 11834
10767 out of 11834
10768 out of 11834
10769 out of 11834
10770 out of 11834
10771 out of 11834
10772 out of 11834
10773 out of 11834
10774 out of 11834
10775 out of 11834
10776 out of 11834
10777 out of 11834
10778 out of 11834
10779 out of 11834
10780 out of 11834
10781 out of 11834
10782 out of 11834
10783 out of 11834
10784 out of 11834
10785 out of 11834
10786 out of 11834
10787 out of 11834
10788 out of 11834
10789 out of

11159 out of 11834
11160 out of 11834
11161 out of 11834
11162 out of 11834
11163 out of 11834
11164 out of 11834
error:  2-(1-Hexyloxyethyl)-2-devinyl pyropheophorbide-a
11165 out of 11834
11166 out of 11834
11167 out of 11834
11168 out of 11834
11169 out of 11834
11170 out of 11834
11171 out of 11834
11172 out of 11834
11173 out of 11834
11174 out of 11834
11175 out of 11834
11176 out of 11834
11177 out of 11834
11178 out of 11834
11179 out of 11834
11180 out of 11834
11181 out of 11834
11182 out of 11834
11183 out of 11834
11184 out of 11834
11185 out of 11834
11186 out of 11834
11187 out of 11834
11188 out of 11834
11189 out of 11834
11190 out of 11834
11191 out of 11834
11192 out of 11834
11193 out of 11834
11194 out of 11834
11195 out of 11834
11196 out of 11834
11197 out of 11834
11198 out of 11834
11199 out of 11834
11200 out of 11834
11201 out of 11834
11202 out of 11834
11203 out of 11834
11204 out of 11834
11205 out of 11834
11206 out of 11834
11207 out of 11834
11208 out of

11585 out of 11834
11586 out of 11834
11587 out of 11834
11588 out of 11834
11589 out of 11834
11590 out of 11834
11591 out of 11834
error:  2-(5-fluoro-pentyl)-2-methyl-malonic-acid
11592 out of 11834
11593 out of 11834
11594 out of 11834
11595 out of 11834
11596 out of 11834
11597 out of 11834
11598 out of 11834
11599 out of 11834
11600 out of 11834
11601 out of 11834
11602 out of 11834
11603 out of 11834
11604 out of 11834
11605 out of 11834
11606 out of 11834
11607 out of 11834
11608 out of 11834
11609 out of 11834
11610 out of 11834
11611 out of 11834
11612 out of 11834
11613 out of 11834
11614 out of 11834
11615 out of 11834
11616 out of 11834
11617 out of 11834
11618 out of 11834
11619 out of 11834
11620 out of 11834
11621 out of 11834
11622 out of 11834
11623 out of 11834
11624 out of 11834
11625 out of 11834
11626 out of 11834
11627 out of 11834
11628 out of 11834
11629 out of 11834
11630 out of 11834
11631 out of 11834
11632 out of 11834
11633 out of 11834
11634 out of 11834


In [13]:
db_all_full_plus_A_10_2 = db_all_full_plus_A_10.reset_index()
MCE_all_full_plus_A_10_2 = MCE_all_full_plus_A_10.reset_index()

# combining columns from DrugBank and MedChemExpress DataFrames
db_MCE_all = pd.concat([db_all_full_plus_A_10_2,MCE_all_full_plus_A_10_2], axis=1)

In [14]:
# extract drugs with logP > 3 & pKa of strongest basic group greater than 4 and less than 11 & physiological charge of >= 0
db_MCE_all_LPC = db_MCE_all  [  (db_MCE_all["logP"].astype(float)>3) 
                              & (db_MCE_all["cb_pKa"].astype(float)>4) 
                              & (db_MCE_all["cb_pKa"].astype(float)<11) 
                              & (db_MCE_all["Phys Chg"].astype(float)>=0)]
# print selected drugs
db_MCE_all_LPC

,level_0,index,drugbank_id,cas,name,type,groups,categories,MW,logP,...,CAS Number,M. Wt,Target,Pathway,Biological Activity,Formula,Solubility,Concentration,Research Area,Clinical Information
75,102,195,DB00206,50-55-5,Reserpine,small molecule,"[approved, investigational]","[Adrenergic Agents, Adrenergic Uptake Inhibito...",608.6787,4.05,...,50-55-5,608.68,Autophagy; Monoamine Transporter,Autophagy; Membrane Transporter/Ion Channel,Reserpine is an inhibitor of the vesicular mon...,C33H40N2O9,DMSO : 25 mg/mL (41.07 mM; Need ultrasonic),10mM,Neurological Disease; Cancer,Launched
76,103,196,DB00207,83905-01-5,Azithromycin,small molecule,[approved],"[Anti-Bacterial Agents, Anti-Infective Agents,...",748.9845,3.03,...,83905-01-5,748.98,Antibiotic; Autophagy; Bacterial,Anti-infection; Autophagy,Azithromycin is a macrolide antibiotic useful ...,C38H72N2O12,DMSO : ≥ 100 mg/mL (133.51 mM),10mM,Infection; Cancer,Launched
77,104,197,DB00208,55142-85-3,Ticlopidine,small molecule,[approved],"[Antiplatelet agents, Blood and Blood Forming ...",263.786,4.25,...,53885-35-1,300.25,Adenosine Receptor,GPCR/G Protein,Ticlopidine hydrochloride is an adenosine diph...,C14H15Cl2NS,DMSO : 50 mg/mL (166.53 mM; Need ultrasonic); ...,10mM,Cardiovascular Disease,Launched
83,111,204,DB00215,59729-33-8,Citalopram,small molecule,[approved],"[Amines, Antidepressive Agents, Antidepressive...",324.3919,3.58,...,59729-32-7,405.30,Autophagy; Serotonin Transporter,Autophagy; Neuronal Signaling,Citalopram hydrobromide is a selective seroton...,C20H22BrFN2O,DMSO : ≥ 38 mg/mL (93.76 mM); H2O : 10 mg/mL (...,10mM,Neurological Disease; Cancer,Launched
84,112,205,DB00216,143322-58-1,Eletriptan,small molecule,"[approved, investigational]","[Agents that produce hypertension, Amines, Ana...",382.519,3.84,...,177834-92-3,463.43,5-HT Receptor,GPCR/G Protein; Neuronal Signaling,Eletriptan HBr is a selective 5-HT1B and 5-HT1...,C22H27BrN2O2S,DMSO : 110 mg/mL (237.36 mM; Need ultrasonic),10mM,Neurological Disease,Launched
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3263,11789,14190,DB16299,1353552-97-2,Poseltinib,small molecule,[investigational],[],470.533,4.54,...,1353552-97-2,470.52,BMX Kinase; Btk; Toll-like Receptor (TLR),Immunology/Inflammation; Protein Tyrosine Kina...,"Poseltinib, an orally active, selective and ir...",C26H26N6O3,DMSO : 100 mg/mL (212.53 mM; Need ultrasonic),10mM,Metabolic Disease,Phase 2
3265,11797,14199,DB16308,199463-33-7,Revaprazan,small molecule,[investigational],"[Heterocyclic Compounds, Fused-Ring, Isoquinol...",362.452,5.17,...,178307-42-1,398.90,Bacterial; COX,Anti-infection; Immunology/Inflammation,Revaprazan hydrochloride is a novel acid pump ...,C22H24ClFN4,DMSO : 8.33 mg/mL (20.88 mM; Need ultrasonic);...,10mM,Infection,Launched
3275,11821,14240,DB16349,1314081-53-2,Vicagrel,small molecule,[investigational],"[Acids, Carbocyclic, Antiplatelet agents, Sulf...",379.86,3.67,...,1314081-53-2,379.86,P2Y Receptor,GPCR/G Protein,"Vicagrel, an acetate derivative of Clopidogrel...",C18H18ClNO4S,DMSO : 250 mg/mL (658.14 mM; Need ultrasonic),10mM,Cardiovascular Disease,Phase 2
3276,11822,14242,DB16351,139290-65-6,Volinanserin,small molecule,[investigational],"[Antidepressive Agents, Benzene Derivatives, C...",373.468,3.25,...,139290-65-6,373.46,5-HT Receptor,GPCR/G Protein; Neuronal Signaling,Volinanserin is a potent and selective antagon...,C22H28FNO3,DMSO : 50 mg/mL (133.88 mM; Need ultrasonic),10mM,Neurological Disease,Phase 3


In [15]:
# reading list of predicted aggregators from Reker et al. (doi: 10.1038/s41565-021-00870-y)
agg_db = pd.read_csv('Data_files/drugbank_selfaggs_smiles.tsv',sep='\t')

# defining first set of drugs that meets all chemical criteria
set1 = db_MCE_all_LPC.loc[db_MCE_all_LPC['name'].isin(set(db_MCE_all_LPC["name"].values) & set(agg_db["NAME"].values))]

In [17]:
# defining set of drugs that doesn't meet logP criterion
set2 = db_MCE_all  [  (db_MCE_all["logP"].astype(float)<3) 
                              & (db_MCE_all["cb_pKa"].astype(float)>4) 
                              & (db_MCE_all["cb_pKa"].astype(float)<11) 
                              & (db_MCE_all["Phys Chg"].astype(float)>=0)]
set2 = set2.loc[set2['name'].isin(set(set2["name"].values) & set(agg_db["NAME"].values))]
set2 = set2.sample(n=9)

In [18]:
# defining set of drugs that doesn't meet pKa criterion
set3 = db_MCE_all  [  (db_MCE_all["logP"].astype(float)>3) 
                              & ((db_MCE_all["cb_pKa"].astype(float)<4) 
                              | (db_MCE_all["cb_pKa"].astype(float)>11)) 
                              & (db_MCE_all["Phys Chg"].astype(float)>=0)]
set3 = set3.loc[set3['name'].isin(set(set3["name"].values) & set(agg_db["NAME"].values))]
set3 = set3.sample(n=9)

In [19]:
# defining set of drugs that doesn't meet Phys Chg criterion
set4 = db_MCE_all  [  (db_MCE_all["logP"].astype(float)>3) 
                              & (db_MCE_all["cb_pKa"].astype(float)>4) 
                              & (db_MCE_all["cb_pKa"].astype(float)<11) 
                              & (db_MCE_all["Phys Chg"].astype(float)<0)]
set4 = set4.loc[set4['name'].isin(set(set4["name"].values) & set(agg_db["NAME"].values))]
set4 = set4.sample(n=5)

In [20]:
# defining set of drugs that doesn't meet predicted aggregator criterion
set5 = db_MCE_all  [  (db_MCE_all["logP"].astype(float)>3) 
                              & (db_MCE_all["cb_pKa"].astype(float)>4) 
                              & (db_MCE_all["cb_pKa"].astype(float)<11) 
                              & (db_MCE_all["Phys Chg"].astype(float)>=0)]
set5 = set5.loc[~set5['name'].isin(set(set5["name"].values) & set(agg_db["NAME"].values))]
set5 = set5.sample(n=9)

In [21]:
# defining set of drugs that doesn't meet predicted aggregator criterion
set6 = db_MCE_all  [  (db_MCE_all["logP"].astype(float)<3) 
                              & ((db_MCE_all["cb_pKa"].astype(float)<4) 
                              | (db_MCE_all["cb_pKa"].astype(float)>11)) 
                              & (db_MCE_all["Phys Chg"].astype(float)<0)]
set6 = set6.loc[~set6['name'].isin(set(set6["name"].values) & set(agg_db["NAME"].values))]
set6=set6.sample(n=8)

In [22]:
set_all = pd.concat([set1,set2,set3,set4,set5,set6],axis=0)

# File export

In [23]:
# Exported on October 18, 2022 
#    due to random selection of drugs for control groups, this file may be different upon rerunning this code
set_all.to_csv('Data_files/MCE_all_drugs_1.csv',index = False)